# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import preprocessing
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


from sqlalchemy import create_engine

In [4]:
# load data from database
engine = create_engine('sqlite:///MessageDatabase.db')
df = pd.read_sql_table("MessagesTable", con=engine)
X = df['message']
y = df.iloc[:,4:]

In [5]:
labels = df.loc[:,"related":].columns

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    #normalize - Remove punctuation characters
    regex = '[^a-zA-Z0-9]'
    detect_text = re.findall(regex, text)

    #tokenize - split text into words
    for list in text:
        list = text.replace(regex," ")
        tokens = word_tokenize(list)

    # Remove Stopwords
    words = [word for word in tokens if word not in stopwords.words('english')]

    #lemanize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []

    for word in words:
        clean_tok = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [11]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.73% 



In [12]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.70      0.34      0.46      1486
           1       0.83      0.96      0.89      5027
           2       0.64      0.17      0.27        41

    accuracy                           0.81      6554
   macro avg       0.72      0.49      0.54      6554
weighted avg       0.80      0.81      0.79      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5436
           1       0.84      0.49      0.62      1118

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6212
           1       0.78      0.11      0.20       342

    accuracy                           0.95      6554
   macro avg       0.87      0.56      0.59      6554
weighted avg       0.94      0.95      0.93      6554

------------------------------------------------------

FEATURE: electricity

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6408
           1       1.00      0.03      0.05       146

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98      0.98      0.97      6554

------------------------------------------------------

FEATURE: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accuracy                    

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [20],
    'vect__ngram_range': [(1, 1)]
}

cv = GridSearchCV(estimator=pipeline, 
                  param_grid=parameters, 
                  scoring='f1_macro', 
                  cv=None, 
                  n_jobs=-1,
                  verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def load_data():
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table("MessagesTable", engine)
    X = df.message.values
    y = df.loc[:,"related":].values 
    return X, y

In [ ]:
def display_results(cv, y_test, y_pred):
    y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
    for column in y_test.columns:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))
        
    print("\nBest Parameters:", cv.best_params_)

In [ ]:
def main(model):
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    display_results(model, y_test, y_pred)

In [ ]:
main(cv)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.